In [1]:
import pandas as pd
import glob
import time
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import np_utils
import keras

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import sys
import numpy as np
sys.path.append('..')
from multivar_lstm.models import baseline_model
from multivar_lstm.lstm_data_generator import get_lstm_sub_data

In [3]:
from gensim.models import Word2Vec

In [4]:
def strtuple2arr(x):
    return np.array(x.replace('(', '').replace(')', '').split(','), dtype=np.float32)

In [5]:
modelpath = "./multivar_lstm/movie_w2v_mincount_model.model"

In [6]:
def get_lstm_sub_data(df, modelpath):
    size = len(df)//1000 * 1000
    df = df.iloc[:size]
    w2v_model = Word2Vec.load(modelpath)

    result_X = []
    Y = 0

    size = len(df)
    for col in df.columns:
        if 'MOVIE_ID' in col:
            if '(t-10)' in col:
                #print(col)
                Y = df[col].values.astype(int).astype(str)
                continue
                
            #print(col)
            arr = w2v_model.wv[df[col].astype(int).astype(str).tolist()]
            result_X.append(arr)

        if 'scaled_duration' in col:
            #print(col)
            arr = df[col].values.reshape(size, 1)
            result_X.append(arr)

        if 'diff_day' in col:
            #print(col)
            arr = df[col].values.reshape(size, 1)
            result_X.append(arr)

        if 'week_coord' in col:
            #print(col)
            arr = np.array(df[col].apply(strtuple2arr).tolist())
            result_X.append(arr)

        if 'day_coord' in col:
            #print(col)
            arr = np.array(df[col].apply(strtuple2arr).tolist())
            result_X.append(arr)

        if 'month_coord' in col:
            #print(col)
            arr = np.array(df[col].apply(strtuple2arr).tolist())
            result_X.append(arr)

        
    X = np.concatenate(result_X, axis=1)
    X = X.reshape(X.shape[0], 10, 108)

    # encode class values as integers
    encoder = OneHotEncoder(sparse=False)
    # get movie all list
    movie_arr = pd.read_csv('multivar_lstm/data/KISA_TBC_MOVIES.tsv',sep='\t' ,encoding='utf-8')['MOVIE_ID'].apply(lambda x: str(x).replace(',', '')).values

    encoder.fit(movie_arr.reshape(-1,1))
    encoded_Y = encoder.transform(Y.reshape(Y.shape[0], 1))
    #print(encoded_Y.shape)
    # convert integers to dummy variables (i.e. one hot encoded)

    return X, encoded_Y

In [7]:
dirpath = 'multivar_lstm/split_data/lstm_dataset/'
modelpath = 'multivar_lstm//movie_w2v_final_model.model'
file_list = glob.glob(dirpath+'/*')
model = baseline_model(shape=(1000, 10, 108), batch_size=1000)
epochs = int(10)
batch_size = 1000

rate = int(len(file_list)*1)

train_file_list = file_list[:rate]

stack_cnt = 0
stack_df = []
for epoch in range(epochs):
    st = time.time()
    
    for train_file in train_file_list:
        if stack_cnt < 3:
            temp = pd.read_csv(train_file,index_col='Unnamed: 0')
            stack_df.append(temp)
            stack_cnt = len(stack_df)
            continue
        
        df = pd.concat(stack_df, axis=0)
        stack_df = []
        stack_cnt = 0
        
        print('{} / {} {}'.format(epoch, epochs, df.info()))
        
        train_X, train_y = get_lstm_sub_data(df, modelpath)
        print(train_X.shape, train_y.shape)
        model.fit(train_X, train_y, batch_size=batch_size, epochs=1)
        print()
    et = time.time()
    print(epoch,'is done time needed ',et - st ,'sec')

model_json = model.to_json()
with open("model/final_model.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("final_model.h5")
print("saved model")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351110 entries, 0 to 117191
Data columns (total 61 columns):
MOVIE_ID(t-0)           351110 non-null int64
scaled_duration(t-0)    351110 non-null float64
diff_day(t-0)           351110 non-null float64
week_coord(t-0)         351110 non-null object
day_coord(t-0)          351110 non-null object
month_coord(t-0)        351110 non-null object
MOVIE_ID(t-1)           351110 non-null float64
scaled_duration(t-1)    351110 non-null float64
diff_day(t-1)           351110 non-null float64
week_coord(t-1)         351110 non-null object
day_coord(t-1)          351110 non-null object
month_coord(t-1)        351110 non-null object
MOVIE_ID(t-2)           351110 non-null float64
scaled_duration(t-2)    351110 non-null float64
diff_day(t-2)           351110 non-null float64
week_coord(t-2)         351110 non-null object
day_coord(t-2)          351110 non-null object
month_coord(t-2)        351110 non-null object
MOVIE_ID(t-3)           351110 non-n

(353000, 10, 108) (353000, 8259)
Epoch 1/1
353000/353000 [==============================] - 67s 190us/step - loss: 3.9431 - acc: 0.3679

<class 'pandas.core.frame.DataFrame'>
Int64Index: 358028 entries, 0 to 119396
Data columns (total 61 columns):
MOVIE_ID(t-0)           358028 non-null int64
scaled_duration(t-0)    358028 non-null float64
diff_day(t-0)           358028 non-null float64
week_coord(t-0)         358028 non-null object
day_coord(t-0)          358028 non-null object
month_coord(t-0)        358028 non-null object
MOVIE_ID(t-1)           358028 non-null float64
scaled_duration(t-1)    358028 non-null float64
diff_day(t-1)           358028 non-null float64
week_coord(t-1)         358028 non-null object
day_coord(t-1)          358028 non-null object
month_coord(t-1)        358028 non-null object
MOVIE_ID(t-2)           358028 non-null float64
scaled_duration(t-2)    358028 non-null float64
diff_day(t-2)           358028 non-null float64
week_coord(t-2)         358028 non-null 

(355000, 10, 108) (355000, 8259)
Epoch 1/1
355000/355000 [==============================] - 65s 183us/step - loss: 3.4214 - acc: 0.4764

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359312 entries, 0 to 118384
Data columns (total 61 columns):
MOVIE_ID(t-0)           359312 non-null int64
scaled_duration(t-0)    359312 non-null float64
diff_day(t-0)           359312 non-null float64
week_coord(t-0)         359312 non-null object
day_coord(t-0)          359312 non-null object
month_coord(t-0)        359312 non-null object
MOVIE_ID(t-1)           359312 non-null float64
scaled_duration(t-1)    359312 non-null float64
diff_day(t-1)           359312 non-null float64
week_coord(t-1)         359312 non-null object
day_coord(t-1)          359312 non-null object
month_coord(t-1)        359312 non-null object
MOVIE_ID(t-2)           359312 non-null float64
scaled_duration(t-2)    359312 non-null float64
diff_day(t-2)           359312 non-null float64
week_coord(t-2)         359312 non-null 

(353000, 10, 108) (353000, 8259)
Epoch 1/1
353000/353000 [==============================] - 62s 174us/step - loss: 3.1686 - acc: 0.5317

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359788 entries, 0 to 120490
Data columns (total 61 columns):
MOVIE_ID(t-0)           359788 non-null int64
scaled_duration(t-0)    359788 non-null float64
diff_day(t-0)           359788 non-null float64
week_coord(t-0)         359788 non-null object
day_coord(t-0)          359788 non-null object
month_coord(t-0)        359788 non-null object
MOVIE_ID(t-1)           359788 non-null float64
scaled_duration(t-1)    359788 non-null float64
diff_day(t-1)           359788 non-null float64
week_coord(t-1)         359788 non-null object
day_coord(t-1)          359788 non-null object
month_coord(t-1)        359788 non-null object
MOVIE_ID(t-2)           359788 non-null float64
scaled_duration(t-2)    359788 non-null float64
diff_day(t-2)           359788 non-null float64
week_coord(t-2)         359788 non-null 

(354000, 10, 108) (354000, 8259)
Epoch 1/1
354000/354000 [==============================] - 66s 187us/step - loss: 2.9598 - acc: 0.5808

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359148 entries, 0 to 118777
Data columns (total 61 columns):
MOVIE_ID(t-0)           359148 non-null int64
scaled_duration(t-0)    359148 non-null float64
diff_day(t-0)           359148 non-null float64
week_coord(t-0)         359148 non-null object
day_coord(t-0)          359148 non-null object
month_coord(t-0)        359148 non-null object
MOVIE_ID(t-1)           359148 non-null float64
scaled_duration(t-1)    359148 non-null float64
diff_day(t-1)           359148 non-null float64
week_coord(t-1)         359148 non-null object
day_coord(t-1)          359148 non-null object
month_coord(t-1)        359148 non-null object
MOVIE_ID(t-2)           359148 non-null float64
scaled_duration(t-2)    359148 non-null float64
diff_day(t-2)           359148 non-null float64
week_coord(t-2)         359148 non-null 

...

(356000, 10, 108) (356000, 8259)
Epoch 1/1
356000/356000 [==============================] - 63s 178us/step - loss: 2.3917 - acc: 0.6504

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355811 entries, 0 to 116591
Data columns (total 61 columns):
MOVIE_ID(t-0)           355811 non-null int64
scaled_duration(t-0)    355811 non-null float64
diff_day(t-0)           355811 non-null float64
week_coord(t-0)         355811 non-null object
day_coord(t-0)          355811 non-null object
month_coord(t-0)        355811 non-null object
MOVIE_ID(t-1)           355811 non-null float64
scaled_duration(t-1)    355811 non-null float64
diff_day(t-1)           355811 non-null float64
week_coord(t-1)         355811 non-null object
day_coord(t-1)          355811 non-null object
month_coord(t-1)        355811 non-null object
MOVIE_ID(t-2)           355811 non-null float64
scaled_duration(t-2)    355811 non-null float64
diff_day(t-2)           355811 non-null float64
week_coord(t-2)         355811 non-null 

(358000, 10, 108) (358000, 8259)
Epoch 1/1
358000/358000 [==============================] - 64s 178us/step - loss: 2.3899 - acc: 0.6512

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360717 entries, 0 to 119219
Data columns (total 61 columns):
MOVIE_ID(t-0)           360717 non-null int64
scaled_duration(t-0)    360717 non-null float64
diff_day(t-0)           360717 non-null float64
week_coord(t-0)         360717 non-null object
day_coord(t-0)          360717 non-null object
month_coord(t-0)        360717 non-null object
MOVIE_ID(t-1)           360717 non-null float64
scaled_duration(t-1)    360717 non-null float64
diff_day(t-1)           360717 non-null float64
week_coord(t-1)         360717 non-null object
day_coord(t-1)          360717 non-null object
month_coord(t-1)        360717 non-null object
MOVIE_ID(t-2)           360717 non-null float64
scaled_duration(t-2)    360717 non-null float64
diff_day(t-2)           360717 non-null float64
week_coord(t-2)         360717 non-null 

(350000, 10, 108) (350000, 8259)
Epoch 1/1
350000/350000 [==============================] - 66s 190us/step - loss: 2.3750 - acc: 0.6534

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357478 entries, 0 to 119386
Data columns (total 61 columns):
MOVIE_ID(t-0)           357478 non-null int64
scaled_duration(t-0)    357478 non-null float64
diff_day(t-0)           357478 non-null float64
week_coord(t-0)         357478 non-null object
day_coord(t-0)          357478 non-null object
month_coord(t-0)        357478 non-null object
MOVIE_ID(t-1)           357478 non-null float64
scaled_duration(t-1)    357478 non-null float64
diff_day(t-1)           357478 non-null float64
week_coord(t-1)         357478 non-null object
day_coord(t-1)          357478 non-null object
month_coord(t-1)        357478 non-null object
MOVIE_ID(t-2)           357478 non-null float64
scaled_duration(t-2)    357478 non-null float64
diff_day(t-2)           357478 non-null float64
week_coord(t-2)         357478 non-null 

(363000, 10, 108) (363000, 8259)
Epoch 1/1
363000/363000 [==============================] - 63s 174us/step - loss: 2.3850 - acc: 0.6517

<class 'pandas.core.frame.DataFrame'>
Int64Index: 358368 entries, 0 to 118304
Data columns (total 61 columns):
MOVIE_ID(t-0)           358368 non-null int64
scaled_duration(t-0)    358368 non-null float64
diff_day(t-0)           358368 non-null float64
week_coord(t-0)         358368 non-null object
day_coord(t-0)          358368 non-null object
month_coord(t-0)        358368 non-null object
MOVIE_ID(t-1)           358368 non-null float64
scaled_duration(t-1)    358368 non-null float64
diff_day(t-1)           358368 non-null float64
week_coord(t-1)         358368 non-null object
day_coord(t-1)          358368 non-null object
month_coord(t-1)        358368 non-null object
MOVIE_ID(t-2)           358368 non-null float64
scaled_duration(t-2)    358368 non-null float64
diff_day(t-2)           358368 non-null float64
week_coord(t-2)         358368 non-null 

KeyboardInterrupt: 

In [8]:
eval_df = pd.read_csv(train_file_list[-1],index_col='Unnamed: 0')
train_X, train_y = get_lstm_sub_data(eval_df, modelpath)

In [9]:
model.evaluate(train_X, train_y,batch_size=batch_size)

119000/119000 [==============================] - 14s 118us/step


[2.394010039938598, 0.6527731098046824]

In [12]:
model_json = model.to_json()
with open("multivar_lstm/model/final_model.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("multivar_lstm/final_model.h5")
print("saved model")

saved model


In [13]:
!ls ./multivar_lstm/

data			 models.py
final_model.h5		 movie_w2v_final_model.model
lstm_data_generator.py	 movie_w2v_mincount_model.model
lstm_dataset		 nohup.out
make_label.py		 __pycache__
make_lstm_dataset.py	 scaled_KISA_TBC_QUESTION.csv
make_movie_w2v_model.py  split_data
model			 split_large_file.py
model.h5		 train.py


In [ ]:
351000*10*108